# The Battle of Neighborhoods


## Find the best locatoion for cofee shop

<br>

## Introduction

Our client want to open new cofee shop in Riyadh city and he is asking about the best location for his business.
For sure he can change anything after he open the cofee shop but not the location, so we have to help him choose the right location, which is the one with lowest number of cofee shops for less cometitions.

<br>


## Business Challenge


To find the best location for new coffe shop we should focus of what we want to have and what we doen't want to have. we want a place with traffic,coffee-drinker people like college, schooles, malles..etc. and what we doen't want to have is competitors from other coffee shops. 
<br>
I will use Foursquare database to provide our client the best location by clustering the areas by its distance from the center fo the city and find the area with lowset number of cofee shops.
<br>



## Data requirements

For this assignment, I will be utilizing the Foursquare API to pull the following location data in Riyadh

- Venue Name
- Venue ID
- Venue Location
- Venue Category
- venue distance
<br>

I will cluster the areas acording to distance from the center and number of venues category equal to **Cofee Shop**

<br>

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 81kB 7.1MB/s eta 0:00:011
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
Libraries imported.


## 1. Load and Explore the Data

world cities coordinates already available on simplemaps.com website..
I've uploaded the world cities coordinates file to github  
<br>

#### 1.1 first I get Riyadh's latitude and longitude values

In [2]:
cities_df = pd.read_csv('https://raw.githubusercontent.com/erefaie/Coursera_Capstone/master/sa.csv')

cities_df.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Riyadh,24.653837,46.715683,Saudi Arabia,SA,Ar Riyāḑ,primary,4465000.0,4205961.0
1,Jeddah,21.542381,39.197969,Saudi Arabia,SA,Makkah al Mukarramah,NaN,3012000.0,2867446.0
2,Ad Dammām,26.434417,50.103263,Saudi Arabia,SA,Ash Sharqīyah,admin,2054710.0,768602.0
3,Medina,24.468611,39.614167,Saudi Arabia,SA,Al Madīnah al Munawwarah,admin,1010000.0,1010000.0
4,Aţ Ţā’if,21.270278,40.415833,Saudi Arabia,SA,Makkah al Mukarramah,NaN,657282.0,530848.0


Find Riyadh latitude & longitude in the dataframe

In [3]:
RY_cordinates = cities_df.loc[cities_df['city'] == 'Riyadh']
RY_latitude = RY_cordinates['lat'].values[0]
RY_longitude = RY_cordinates['lng'].values[0]


print(RY_latitude)
print(RY_longitude)

24.653837
46.715683


<br>
<br>



Here i am going to start utilizing the Foursquare API to explore what we want to have which is colleges.

#### 1.2 Define Foursquare Credentials, Version, and build the needed URL

In [4]:
CLIENT_ID = '2LUYJPERSOIYVIPLYFOHOGV1ZPASRSOVIUGRQ53ABMNZ3KTU' # My Foursquare ID
CLIENT_SECRET = 'TH4VCMJBMD1ZLN0BSO3OVSABF50Z31VNCNDBZLHBZOBR42QA' # My Foursquare Secret
VERSION = '20180605' # Foursquare API version


#### 1.3 Get the list of categories from Foursquare

In [5]:
# will load  the categories defined in Foursquare in order to get the one related to college and malles

url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION)

categories_results = requests.get(url).json()


In [6]:

ct = categories_results['response']['categories']
    
ct_df = json_normalize(ct) # flatten JSON


# filter columns
filtered_columns = ['id', 'name']
ct_df = ct_df.loc[:, filtered_columns]


# clean columns
ct_df.columns = [col.split(".")[-1] for col in ct_df.columns]

ct_df

,id,name
0,4d4b7104d754a06370d81259,Arts & Entertainment
1,4d4b7105d754a06372d81259,College & University
2,4d4b7105d754a06373d81259,Event
3,4d4b7105d754a06374d81259,Food
4,4d4b7105d754a06376d81259,Nightlife Spot
5,4d4b7105d754a06377d81259,Outdoors & Recreation
6,4d4b7105d754a06375d81259,Professional & Other Places
7,4e67e38e036454776db1fb3a,Residence
8,4d4b7105d754a06378d81259,Shop & Service
9,4d4b7105d754a06379d81259,Travel & Transport


<br>
Now I'll create the GET request URL. to get the venues with **College & University** category using **categoryId** parameter

In [7]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radious={}&categoryId={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
RY_latitude,
RY_longitude,
100,
500,
'4d4b7105d754a06372d81259')

print(url)

https://api.foursquare.com/v2/venues/explore?&client_id=2LUYJPERSOIYVIPLYFOHOGV1ZPASRSOVIUGRQ53ABMNZ3KTU&client_secret=TH4VCMJBMD1ZLN0BSO3OVSABF50Z31VNCNDBZLHBZOBR42QA&v=20180605&ll=24.653837,46.715683&limit=100&radious=500&categoryId=4d4b7105d754a06372d81259



#### 1.4 Send the GET request and examine the resutls


In [8]:
results = requests.get(url).json()

Read the resulted information from json file dsinto pandas dataframe.

In [9]:
#load the result to dataframe

uni = results['response']['groups'][0]['items']
    
uni_df = json_normalize(uni) # flatten JSON


# filter columns
filtered_columns = ['venue.name', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance']
uni_df = uni_df.loc[:, filtered_columns]



# clean columns
uni_df.columns = ['Name', 'latitude', 'longitude', 'Distance']

Will use head function to have a look to the data

In [10]:
uni_df.head()


,Name,latitude,longitude,Distance
0,استراحة الطالبات,24.661130,46.718207,851
1,KSU - Statistical Sciences bld.9,24.663049,46.732317,1970
2,King Saud University - Malaz,24.663421,46.732213,1983
3,جامعة الملك سعود,24.644856,46.696366,2195
4,College Of Dentistry (كلية طب الأسنان),24.657858,46.700616,1588


In [11]:


uni_df.shape

(100, 4)


universities dataframe comprise of 100 observations and 4 characteristics

In [12]:
uni_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
Name         100 non-null object
latitude     100 non-null float64
longitude    100 non-null float64
Distance     100 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 3.2+ KB


<br> <br>
The Dataset has object, float64 and int64 values 
<br>
No variable column has num or missing values



In [13]:
# create map of universities/college in Riyadh using latitude and longitude values
map_uni = folium.Map(location=[RY_latitude, RY_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(uni_df['latitude'], uni_df['longitude'], uni_df['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_uni)  
    
map_uni

In [14]:
# load sub sategories of Food category in order to get coffee shop category id

food_category_id = '4d4b7105d754a06374d81259' # from the previous loaded list

url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION)

categories_results = requests.get(url).json()


#readh nested json to get the coffee shop category id
food_categories = categories_results['response']['categories']
food_categories_df = json_normalize(food_categories, record_path=['categories']) # flatten JSON

food_categories_df.loc[food_categories_df['name'] == 'Coffee Shop']

,categories,icon,id,name,pluralName,shortName
95,[],{'prefix': 'https://ss3.4sqi.net/img/categorie...,4bf58dd8d48988d1e0931735,Coffee Shop,Coffee Shops,Coffee Shop


In [15]:
# a function to get the venues with category coffe shops nearby each uni/college

LIMIT = 20
radius = 300
coffee_shops_category_id = '4bf58dd8d48988d1e0931735'

def getNearbyVenue(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            coffee_shops_category_id)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['College', 
                  'CollegeLatitude', 
                  'CollegeLongitude', 
                  'Venue', 
                  'VenueLatitude', 
                  'VenueLongitude', 
                  'VenueCategory']
    
    return(nearby_venues)

In [16]:

Uni_Nearby_CoffeShops = getNearbyVenue(names=uni_df['Name'],
                                   latitudes=uni_df['latitude'],
                                   longitudes=uni_df['longitude']
                                  )


In [17]:
print(Uni_Nearby_CoffeShops.shape)

Uni_Nearby_CoffeShops.head()


(580, 7)


,College,CollegeLatitude,CollegeLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,استراحة الطالبات,24.661130,46.718207,دانكن دونتس مستشفى الملك عبدالعزيز,24.661061,46.718328,Coffee Shop
1,استراحة الطالبات,24.661130,46.718207,dr.CAFE COFFEE | د. كيف,24.659507,46.718243,Coffee Shop
2,استراحة الطالبات,24.661130,46.718207,4Twins Coffee & Sweet,24.661447,46.719114,Café
3,استراحة الطالبات,24.661130,46.718207,perfect cafe 😒,24.662602,46.715489,Café
4,KSU - Statistical Sciences bld.9,24.663049,46.732317,Starbucks (ستاربكس),24.662603,46.729846,Coffee Shop


## 2. Cluster College with nearby Coffee shops

In [29]:

uni_cs_count = Uni_Nearby_CoffeShops.groupby(['College', 'CollegeLatitude', 'CollegeLongitude']).size().to_frame('CF Count').reset_index()
uni_cs_count

,College,CollegeLatitude,CollegeLongitude,CF Count
0,02GA010,24.633398,46.679511,9
1,AU Library (مكتبة جامعة الفيصل),24.665466,46.676914,2
2,Academic Tower KSMC Calss 2,24.629217,46.690792,6
3,Al Thaqafi Center Higher Cultural Institute fo...,24.682908,46.697048,8
4,Alfaisal University (جامعة الفيصل),24.664931,46.676748,8
5,Building 6 (مبنى ٦),24.663244,46.732449,4
6,Building No.3 | KSU,24.631871,46.679717,8
7,Bulding 5 - King Abduaziz University Hospital,24.660911,46.720102,4
8,College Of Applied Medical Sciences (CAMS),24.632779,46.674634,5
9,College Of Arts - English Literature,24.632958,46.680605,8


Sort the dataframe by count of coffee shopes near the college

In [30]:
uni_cs_count = uni_cs_count.sort_values(by=['CF Count'])

<br>
College name has categoral variable so will drop it and run clustering

In [31]:
df = uni_cs_count.drop('College', axis=1)
df.head()

,CollegeLatitude,CollegeLongitude,CF Count
29,24.650254,46.727350,1
82,24.638183,46.692760,1
39,24.645778,46.693649,1
72,24.634670,46.692614,1
80,24.647422,46.694448,1


Run k-means to cluster the areas into 5 clusters

In [32]:
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [33]:
# add clustering labels
uni_cs_count.insert(0, 'Cluster Labels', kmeans.labels_)

uni_cs_count.head()

,Cluster Labels,College,CollegeLatitude,CollegeLongitude,CF Count
29,0,Institute of Saudi Electricity Company,24.650254,46.727350,1
82,0,مدارس التضامن الأهلية,24.638183,46.692760,1
39,0,Prince Saud Al-Faysal Institute of Diplomatic ...,24.645778,46.693649,1
72,0,جلسة مجلس الامن الدولي,24.634670,46.692614,1
80,0,كلية امن الشبكات جامعة الملك سعود,24.647422,46.694448,1


#### As we can see from the above result, the area with less coffee shops is in cluster 0 

In [34]:
uni_cs_count.loc[uni_cs_count['CF Count'] < 4]

,Cluster Labels,College,CollegeLatitude,CollegeLongitude,CF Count
29,0,Institute of Saudi Electricity Company,24.650254,46.727350,1
82,0,مدارس التضامن الأهلية,24.638183,46.692760,1
39,0,Prince Saud Al-Faysal Institute of Diplomatic ...,24.645778,46.693649,1
72,0,جلسة مجلس الامن الدولي,24.634670,46.692614,1
80,0,كلية امن الشبكات جامعة الملك سعود,24.647422,46.694448,1
74,0,طريق جامعة الملك سعود,24.636858,46.689571,1
27,0,"Imam Muhammad Ibn Saud University, Girls Colle...",24.655973,46.736283,1
21,0,Female Lecture Hall,24.664968,46.677316,2
44,0,Riyadh Elm University (REU) (جامعة رياض العلم ...,24.657750,46.700598,2
10,0,College Of Dentistry (كلية طب الأسنان),24.657858,46.700616,2


In [35]:
# create map
map_clusters = folium.Map(location=[RY_latitude, RY_longitude], zoom_start=13)

# set the color according to the number of coffee shops nearby the college
rainbow = [colors.rgb2hex('lime'), colors.rgb2hex('yellow'), colors.rgb2hex('red') ]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(uni_cs_count['CollegeLatitude'], uni_cs_count['CollegeLongitude'], uni_cs_count['College'], uni_cs_count['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### from the result, south west Riyadh is the best place for new coffee shop as it has lots of coffee-drinker people with less compititors